<a href="https://colab.research.google.com/github/wonwookim/DataMining/blob/main/Logistic_Regression_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install dmba

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 19.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd

In [ ]:
bank = pd.read_csv('/content/drive/MyDrive/data_mining/UniversalBank.csv')
bank.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [ ]:
bank.drop(columns = ['ID', 'ZIP Code'], inplace = True)
bank.columns = [c.replace(' ', '_') for c in bank.columns]
bank.head()

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Personal_Loan,Securities_Account,CD_Account,Online,CreditCard
0,25,1,49,4,1.6,1,0,0,1,0,0,0
1,45,19,34,3,1.5,1,0,0,1,0,0,0
2,39,15,11,1,1.0,1,0,0,0,0,0,0
3,35,9,100,1,2.7,2,0,0,0,0,0,0
4,35,8,45,4,1.0,2,0,0,0,0,0,1


In [ ]:
bank['Education'] = bank['Education'].astype('category')
new_categories = {1: 'Undergrad', 2: 'Graduate', 3: 'Advanced/Professional'}
bank.Education.cat.rename_categories(new_categories, inplace = True)
bank = pd.get_dummies(bank, prefix_sep = '__', drop_first = True) #prefix_sep을 이용해서 더미데이터 앞에 어떤 값을 붙이는지?
bank

<ipython-input-11-1b84d05d6ec2>:3: FutureWarning: The `inplace` parameter in pandas.Categorical.rename_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  bank.Education.cat.rename_categories(new_categories, inplace = True)


,Age,Experience,Income,Family,CCAvg,Mortgage,Personal_Loan,Securities_Account,CD_Account,Online,CreditCard,Education__Graduate,Education__Advanced/Professional
0,25,1,49,4,1.6,0,0,1,0,0,0,0,0
1,45,19,34,3,1.5,0,0,1,0,0,0,0,0
2,39,15,11,1,1.0,0,0,0,0,0,0,0,0
3,35,9,100,1,2.7,0,0,0,0,0,0,1,0
4,35,8,45,4,1.0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,29,3,40,1,1.9,0,0,0,0,1,0,0,1
4996,30,4,15,4,0.4,85,0,0,0,1,0,0,0
4997,63,39,24,2,0.3,0,0,0,0,0,0,0,1
4998,65,40,49,3,0.5,0,0,0,0,1,0,1,0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

y = bank['Personal_Loan']
x = bank.drop(columns = ['Personal_Loan'])
train_X, valid_X, train_y, valid_y = train_test_split(x, y, test_size = 0.4, random_state =1)

logit_reg = LogisticRegression(penalty = 'l2', C = 1e42, solver ='liblinear') #penalty 를 이용해 하이퍼파라미터 설정
logit_reg.fit(train_X, train_y)

print('intercept', logit_reg.intercept_[0])
print(pd.DataFrame({'coeff': logit_reg.coef_[0]}, index= x.columns).transpose())

intercept -12.493436061176814
            Age  Experience    Income    Family     CCAvg  Mortgage  \
coeff -0.037685    0.039202  0.058844  0.612251  0.240489  0.001012   

       Securities_Account  CD_Account    Online  CreditCard  \
coeff            -1.01428    3.649097 -0.678306   -0.958283   

       Education__Graduate  Education__Advanced/Professional  
coeff             4.202148                          4.355761  


In [ ]:
logit_reg_pred = logit_reg.predict(valid_X)
logit_reg_prob = logit_reg.predict_proba(valid_X)
logit_result = pd.DataFrame({'actual' : valid_y,
                             'p(0)' : [p[0] for p in logit_reg_prob],
                             'p(1)' : [p[1] for p in logit_reg_prob],
                             'predicted' : logit_reg_pred})
case1 = [702, 932, 2764, 2721]
print(logit_result.loc[case1])

      actual      p(0)      p(1)  predicted
702        1  0.985893  0.014107          0
932        0  0.331000  0.669000          1
2764       0  0.976092  0.023908          0
2721       1  0.031430  0.968570          1


In [ ]:
from dmba import classificationSummary
classificationSummary(train_y, logit_reg.predict(train_X))
classificationSummary(valid_y, logit_reg.predict(valid_X))

Confusion Matrix (Accuracy 0.9600)

       Prediction
Actual    0    1
     0 2683   30
     1   90  197
Confusion Matrix (Accuracy 0.9595)

       Prediction
Actual    0    1
     0 1791   16
     1   65  128


In [ ]:
import matplotlib.pyplot as plt
from dmba import gainsChart

df = logit_result.sort_values(by = ['p(1)'], ascending = False)
gainsChart(df.actual, figsize = (6,4))
gainsChart

<function dmba.graphs.gainsChart(gains: pandas.core.series.Series, color: str = 'C0', label: Optional[str] = None, ax: Any = None, figsize: Any = None) -> Any>

In [ ]:
plt.plot(df.actual)

### Multinomial Logisic Regression

In [ ]:
import statsmodels.api as sm
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/data_mining/AccidentsFull.csv')
data

,HOUR_I_R,ALCHL_I,ALIGN_I,STRATUM_R,WRK_ZONE,WKDY_I_R,INT_HWY,LGTCON_I_R,MANCOL_I_R,PED_ACC_R,...,SUR_COND,TRAF_CON_R,TRAF_WAY,VEH_INVL,WEATHER_R,INJURY_CRASH,NO_INJ_I,PRPTYDMG_CRASH,FATALITIES,MAX_SEV_IR
0,0,2,2,1,0,1,0,3,0,0,...,4,0,3,1,1,1,1,0,0,1
1,1,2,1,0,0,1,1,3,2,0,...,4,0,3,2,2,0,0,1,0,0
2,1,2,1,0,0,1,0,3,2,0,...,4,1,2,2,2,0,0,1,0,0
3,1,2,1,1,0,0,0,3,2,0,...,4,1,2,2,1,0,0,1,0,0
4,1,1,1,0,0,1,0,3,2,0,...,4,0,2,3,1,0,0,1,0,0


In [ ]:
outcome = 'MAX_SEV_IR'
predictors = ['ALCHL_I', 'WEATHER_R']

y = data[outcome]
X = data[predictors]

print('Multinominal Logistic Regression')
logit = sm.MNLogit(y, sm.add_constant(X))
logit_fit = logit.fit()
print(logit_fit.summary())

Multinominal Logistic Regression
Optimization terminated successfully.
         Current function value: 0.743054
         Iterations 9
                          MNLogit Regression Results                          
Dep. Variable:             MAX_SEV_IR   No. Observations:                42183
Model:                        MNLogit   Df Residuals:                    42177
Method:                           MLE   Df Model:                            4
Date:                Mon, 12 Jun 2023   Pseudo R-squ.:                0.004253
Time:                        14:21:43   Log-Likelihood:                -31344.
converged:                       True   LL-Null:                       -31478.
Covariance Type:            nonrobust   LLR p-value:                 9.739e-57
MAX_SEV_IR=1       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.9947      0.075     13.257      0.000       0.848   

In [ ]:
!pip install mord
from sklearn.linear_model import LogisticRegression
from mord import LogisticIT

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for mord: filename=mord-0.7-py3-none-any.whl size=9885 sha256=93b598ec6c55b589d87e08a7f57c082c45ddee3d0d7601c1a23fbf39b870b6b7
  Stored in directory: /root/.cache/pip/wheels/77/00/19/3cea86fbfc737ec4acb515cd94497dcc33f943fa157548b96c
Successfully built mord


In [ ]:
y= data[outcome]
X = data[predictors]
train_X, train_y = X,y

print('Nominal logistic regression')
logit = LogisticRegression(multi_class ='multinomial')
logit.fit(X,y)

print(' intercept', logit.intercept_)
print(' intercept', logit.coef_)
print()
probs = logit.predict_proba(X)
results = pd.DataFrame({'actual' : y,
                        'predicted' : logit.predict(X),
                        'p(0)' : [p[0] for p in probs],
                        'p(1)' : [p[1] for p in probs],
                        'p(2)' : [p[2] for p in probs]})
print(results.loc[25])
print()

Nominal logistic regression
 intercept [-0.08076524  0.91268277 -0.83191753]
 intercept [[ 0.5128394   0.33502408]
 [ 0.14631031  0.09149218]
 [-0.65914971 -0.42651626]]

actual       0.000000
predicted    0.000000
p(0)         0.553388
p(1)         0.441150
p(2)         0.005462
Name: 25, dtype: float64

